In [8]:
import os
os.chdir('C:/Users/RH/A_whole_try_prog/y_test_data')

In [9]:
import pandas as pd
import numpy as np
import esm
import torch

print(pd.__version__)
print(np.__version__)
print(esm.__version__)
print(torch.__version__)

1.5.3
1.22.0
2.0.0
2.0.0


In [10]:
import numpy as np
import pandas as pd

In [11]:
#training dataset loading
#打开指定路径的excel文件
excel_path = r'C:\Users\RH\A_whole_try_prog\X_test.xlsx'
dataset = pd.read_excel(excel_path, na_filter=False)
sequence_list = dataset['Sequence']

In [12]:
print(sequence_list)

0                 HVRETALV
1                      YFE
2                     PHYL
3     IANNEADAISLDGGQVFEAG
4                      CHI
5                      LPL
6                      LQL
7                      YNL
8                    IEEAF
9                     FEQL
10                  SVPQPK
11                   EVPKA
12                  PLAQPA
13                   LLNPT
14                   HLPLP
15                  VLPIPQ
16                     LWL
17                     PWK
18                     PWM
19                     RWD
Name: Sequence, dtype: object


In [13]:
peptide_sequence_list = []
for seq in sequence_list:
    format_seq = [seq,seq]
    tuple_sequence = tuple(format_seq)
    peptide_sequence_list.append(tuple_sequence)
print(peptide_sequence_list)

[('HVRETALV', 'HVRETALV'), ('YFE', 'YFE'), ('PHYL', 'PHYL'), ('IANNEADAISLDGGQVFEAG', 'IANNEADAISLDGGQVFEAG'), ('CHI', 'CHI'), ('LPL', 'LPL'), ('LQL', 'LQL'), ('YNL', 'YNL'), ('IEEAF', 'IEEAF'), ('FEQL', 'FEQL'), ('SVPQPK', 'SVPQPK'), ('EVPKA', 'EVPKA'), ('PLAQPA', 'PLAQPA'), ('LLNPT', 'LLNPT'), ('HLPLP', 'HLPLP'), ('VLPIPQ', 'VLPIPQ'), ('LWL', 'LWL'), ('PWK', 'PWK'), ('PWM', 'PWM'), ('RWD', 'RWD')]


In [14]:
#调试完毕，以此为准！
import torch
import esm
import pandas as pd

#load ESM-2 model
model, alphabet = esm.pretrained.esm2_t30_150M_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval()
    
#load data
data = peptide_sequence_list
batch_labels, batch_strs, batch_tokens = batch_converter(data)
batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
    
# Extract per-residue representations (on CPU)
with torch.no_grad():
    results = model(batch_tokens, repr_layers=[30], return_contacts=True)
token_representations = results["representations"][30]
    
# Generate per-sequence representations via averaging
sequence_representations = []
for i, token_len in enumerate(batch_lens):
    each_seq_rep = token_representations[i, 1:token_len - 1].mean(0).tolist()  
    sequence_representations.append(each_seq_rep)

embedding_results = pd.DataFrame(sequence_representations)
print(embedding_results)
embedding_results.to_csv('X_test_640.csv')

         0         1         2         3         4         5         6    \
0  -0.195990  0.108168  0.235250 -0.103984 -0.153602  0.010032  0.001333   
1  -0.057576  0.135244  0.133541 -0.297181 -0.152541  0.004459 -0.096750   
2  -0.139756 -0.056552  0.106659 -0.292172  0.038612 -0.109640  0.081992   
3  -0.005653  0.109568  0.118261  0.086661 -0.168748  0.016080 -0.003229   
4  -0.243100  0.113281  0.327936 -0.446812  0.063819 -0.017027  0.107023   
5  -0.220327  0.106315  0.226592 -0.283321  0.097029 -0.129272 -0.056065   
6  -0.160703  0.100113  0.191967 -0.264349  0.060579 -0.193532 -0.027650   
7  -0.189151  0.048195  0.073360 -0.314162 -0.103760 -0.165018 -0.049925   
8  -0.099521  0.121308  0.170791 -0.172098 -0.188271 -0.055555 -0.072078   
9  -0.077919  0.104360  0.099158 -0.202731 -0.051281 -0.145980 -0.047873   
10 -0.169235 -0.018591  0.183626 -0.112396 -0.187161 -0.048722  0.012766   
11 -0.113216 -0.013956  0.241991 -0.037804 -0.244806 -0.013096 -0.023129   
12 -0.135004